In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import plotly.express as px
import plotly.graph_objs as go

pump_times_filename = Path.home() / 'Documents' / 'Tehaleh' / 'CDF pump run times.xlsx'
df = pd.read_excel(pump_times_filename)
df_columns = pd.Index(
    ['Date', '1A', '1B', '2A', '2B', '3A', '3B',
     '4A', '4B', '5A', '5B']
)
df.columns = df_columns
df = df.set_index('Date')
df_rates = pd.read_excel(pump_times_filename, 1)
df_rates = df_rates.set_index('pump')

for lobe in range(1, 6):
    pump_A = f'{lobe}A'
    pump_B = f'{lobe}B'
    lobeA = f'{lobe}A'
    lobeB = f'{lobe}B'
    more_zero_A = (df[lobeA] > 0)
    more_zero_B = (df[lobeB] > 0)
    is_zero_A = (df[lobeA] == 0)
    is_zero_B = (df[lobeB] == 0)
    more_zero_both = more_zero_A & more_zero_B
    diff_b_a = df[lobeB] - df[lobeA]
    just_A = more_zero_A & is_zero_B
    just_B = more_zero_B & is_zero_A

    flow_cases = [
        diff_b_a.loc[df.loc[more_zero_both, lobeA:lobeB].loc[diff_b_a > 0].index] * df_rates.loc[lobeB][0],
        diff_b_a.loc[df.loc[more_zero_both, lobeA:lobeB].loc[diff_b_a < 0].index] * df_rates.loc[lobeA][0] * -1,
        df.loc[more_zero_both, lobeA:lobeB].loc[diff_b_a > 0].loc[:, lobeA] * df_rates.loc['2_pumps'][0],
        df.loc[more_zero_both, lobeA:lobeB].loc[diff_b_a < 0].loc[:, lobeB] * df_rates.loc['2_pumps'][0],
        df.loc[just_B, lobeB] * df_rates.loc[lobeB][0],
        df.loc[just_A, lobeA] * df_rates.loc[lobeA][0]
    ]
    flow_cases[0].name = lobeB
    flow_cases[1].name = lobeA
    
    for flows in flow_cases:
        df.update(flows)
        
df_to_concat = []
for lobe in range(1, 6):
    lobeA = f'{lobe}A'
    lobeB = f'{lobe}B'
    df1 = df[lobeA] + df[lobeB]
    df1.name = lobe
    df_to_concat += [df1]
df_flows = pd.concat(df_to_concat, axis=1)
df_percentages = df_flows.div(df_flows.sum(axis=1), axis=0)
df_percentages.columns = ('1_per', '2_per', '3_per', '4_per', '5_per')
df_final = pd.concat([df_flows, df_percentages], axis=1)
fig = go.Figure()
fig.update_layout(
    yaxis2=dict(
        title="Percentage of total flow",
        overlaying="y",
        side="right"
    ),
    yaxis_title='Flow (gpd)',
    dragmode='pan'
)
for col in df_final:
    if 'per' in str(df_final[col].name):
        y_axis = 'y2'
        line_mode = 'dash'
    else:
        y_axis = 'y1'
        line_mode = 'solid'
    fig.add_scattergl(
        x=df_final[col].index,
        y=df_final[col].tolist(),
        yaxis=y_axis,
        name=df_final[col].name,
        line_dash = line_mode
    )
    

In [2]:
fig.show(renderer='browser', config={'scrollZoom': True})

In [73]:
fig.write_html('run_times.html', config={'scrollZoom': True}
)

In [65]:
df

,1A,1B,2A,2B,3A,3B,4A,4B,5A,5B
Date,,,,,,,,,,
2023-11-11,5580.8,98862.5,130.8,103344.6,56628.0,0.0,92180.4,3488.0,0.0,38025.6
2023-11-12,0.0,111675.0,0.0,109691.4,59488.0,0.0,101300.4,0.0,0.0,40617.6
2023-11-13,0.0,107600.0,0.0,104943.6,56589.0,174.4,97458.6,0.0,0.0,39168.0
2023-11-14,0.0,100062.5,0.0,97674.3,54899.0,0.0,90003.0,0.0,0.0,33494.4
2023-11-15,0.0,101075.0,0.0,99211.8,47593.0,0.0,91154.4,0.0,0.0,41001.6
2023-11-16,0.0,100362.5,0.0,98043.3,50700.0,0.0,91086.0,0.0,0.0,38102.4
2023-11-17,42466.4,40662.5,38891.2,34513.8,13260.0,47916.4,50787.0,28579.8,44166.8,18163.2
2023-11-18,0.0,87675.0,4673.6,71351.4,67710.8,4708.5,77736.6,0.0,3149.4,67972.4
2023-11-19,0.0,96175.0,9232.9,74948.4,72027.2,9120.3,85990.2,0.0,6351.0,72005.4
